In [7]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
#from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
X_train = load('trainXgyr.npy')
X_test = load('testXgyr.npy')
y_train = load('trainygyr.npy')
y_test = load('testygyr.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [9]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

# 1-LSTM

In [10]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [11]:
model1 = Sequential()
#forward_layer =LSTM(500,input_shape=(n_timesteps,n_features),return_sequences=True)
#backward_layer =LSTM(500,activation='relu',return_sequences=True,go_backwards=True)
#model1.add(Bidirectional(LSTM(500, input_shape=(n_timesteps,n_features))))
#model1.add(Bidirectional(LSTM(500, input_shape=(n_timesteps,n_features))))
model1.add(Bidirectional(LSTM(256, return_sequences=False), input_shape=(n_timesteps,n_features)))
#model1.add(Bidirectional(LSTM(500)))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(128, activation='relu'))
#model1.add(Dropout(0.1))
#model1.add(Dense(100, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               532480    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 118)               15222     
Total params: 711,926
Trainable params: 711,926
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model1.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/200
207/207 [==============================] - 345s 1s/step - loss: 2.3397 - accuracy: 0.4627 - val_loss: 1.0648 - val_accuracy: 0.7349
Epoch 2/200
207/207 [==============================] - 333s 2s/step - loss: 0.8208 - accuracy: 0.7955 - val_loss: 0.6413 - val_accuracy: 0.8345
Epoch 3/200
207/207 [==============================] - 333s 2s/step - loss: 0.5290 - accuracy: 0.8641 - val_loss: 0.4698 - val_accuracy: 0.8836
Epoch 4/200
207/207 [==============================] - 357s 2s/step - loss: 0.3922 - accuracy: 0.8954 - val_loss: 0.3786 - val_accuracy: 0.9023
Epoch 5/200
207/207 [==============================] - 367s 2s/step - loss: 0.3231 - accuracy: 0.9126 - val_loss: 0.4076 - val_accuracy: 0.8914
Epoch 6/200
207/207 [==============================] - 333s 2s/step - loss: 0.2620 - accuracy: 0.9287 - val_loss: 0.3255 - val_accuracy: 0.9198
Epoch 7/200
207/207 [==============================] - 333s 2s/step - loss: 0.2108 - accuracy: 0.9415 - val_loss: 0.3022 - val_accuracy:

In [15]:
loss, accuracy = model1.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9378
Testing Accuracy: 0.8778
